In [1]:
try:
    import spacy
except:
    import spacy
import json
from datetime import datetime
name = 'finance_course1'
JSON_path = '/home/ubuntu/Questions_generation/Financial Markets Course 2.json'
sections_num_max= 'all'
%pdb on


/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Automatic pdb calling has been turned ON


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
#questions
fig = mcq.plot_similarity_matrix((questions_df[questions_used]).to_list())
fig.update_layout(
    autosize=False,
    width=2000,
    height=2000)
fig.show()
#both
fig = mcq.plot_similarity_matrix((questions_df[questions_used]+' '+questions_df['selected_ans']).to_list())
fig.update_layout(
    autosize=False,
    width=2000,
    height=2000)

In [2]:
from MCQ import flanT5MCQ
from datetime import datetime
generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}

answers_generator_args = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": 8,#10
    "length_penalty":0.2,
    #"length_penalty": 1.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[tokenizer.encode(['.'])],
    'top_p' :0.97,
    'diversity_penalty':float(8),
    'num_beam_groups':8,#10,
    "return_dict_in_generate" :True,
    'output_scores':True,
    "early_stopping": True,
    'num_return_sequences':5
}
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
dir_path = '../outputs/'+datetime.now().strftime("%d_%m_%y")+'_'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open(JSON_path) as f:
  result = json.load(f)

sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
if sections_num_max!='all':
  sections = sections[:sections_num_max]
print(f'Limit questions to be from the {sections_num_max} first sections')
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]
min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
short_answers_generator_args = copy.deepcopy(answers_generator_args)
short_answers_generator_args["length_penalty"]=-0.6
if 'mcq' in locals():
  del mcq
mcq = flanT5MCQ(generator_args=generator_args,answers_generator_args=answers_generator_args,short_answers_generator_args=short_answers_generator_args)
'''
cpu_model = mcq.model.to('cpu')
del mcq.model
mcq.model = cpu_model
mcq.push_model_to_GPU(mcq.model)
'''
with torch.no_grad():
  questions_df = mcq.generate_questions(sections=sections,org_sections=org_text,sections_ranks=np.ones(len(sections)))
questions_df.to_pickle(dir_path+'GQ.pickle')
print(f"Stage 1:{dir_path+'GQ.pickle'} has been saved")
torch.cuda.empty_cache()
if mcq.COMPUTE_ANSWERS:
  mcq.push_model_to_GPU(mcq.QA.model)
  questions_df = mcq.QA.select_best_answer(questions_df)
  questions_df.to_pickle(dir_path+'QG+QA.pickle')
  print(f"Stage 2:{dir_path+'QG+QA.pickle'} has been saved")
  questions_df = mcq.QG.create_question_MixQG(questions_df)
  questions_df.to_pickle(dir_path+'GQ+QA+GQ.pickle')
  print(f"Stage 3:{dir_path+'GQ+QA+GQ.pickle'} has been saved")
  questions_used = 'new_question'
  questions_df['RQUGE'] = questions_df.apply(lambda x: mcq.rquge.scorer(x.text, x[questions_used], x.selected_ans)[0]['pred_score'] ,axis='columns')
  questions_df = questions_df.sort_values('RQUGE',ascending=False).reset_index()
  q_sim_mat,q_ans_sim_mat = mcq.find_similarity(questions_df[questions_used].to_list(),
                                            answers = (questions_df[questions_used]+' '+questions_df['selected_ans']).to_list())
  filter_idx = mcq.filter_questions(questions_df[questions_used].to_list(),
                                  q_sim_mat = q_sim_mat, ans_sim_mat = q_ans_sim_mat,
                                  n_thrs=20, return_index=True)
  questions_df['use_question']=False
  questions_df.loc[filter_idx,'use_question']=True
  #questions_df = questions_df.iloc[filter_idx,:]
with open(f'{dir_path}questions_full.txt', 'w') as f:
    qs_text =[mcq.show_qs(questions_df,i) for i in questions_df.index.values]
    text_outuput=''
    for i,qs in zip(questions_df.index.values,qs_text):
        text_outuput += f"({i})TAKEN?{questions_df['use_question'][i]} RQUGE:{round(questions_df['RQUGE'][i],4)}"+'\n'+ qs+"\n\n"
    f.write(text_outuput)
    print(f"Stage 4:{dir_path}questions_full.txt has been saved")
    
with open(f'{dir_path}questions.txt', 'w') as f:
    text_outuput=''
    for i in np.unique(questions_df.section_n_chunk):
      qs_in_sections = questions_df.query(f'section_n_chunk == {i} and use_question == True')
      for q,a in zip(qs_in_sections[questions_used],qs_in_sections.selected_ans):
        text_outuput+=f'Q:{q}\nA:{a}\n'
      text_outuput+= '-'*50 + '\n'
    f.write(text_outuput)
    print(f"Stage 5:{dir_path}questions.txt has been saved")

Limit questions to be from the all first sections


Token indices sequence length is longer than the specified maximum sequence length for this model (963 > 512). Running this sequence through the model will result in indexing errors


Abbreviation 	 Definition
{}


0it [00:00, ?it/s]

Total text shape is torch.Size([1, 376])
Qs:Who were some of these institutions that collapsed?
Qs:The speaker is going to talk about what in the context?
Qs:What happened after the stock market collapsed
Qs:How did this lecture begin?
Qs:When the stock market collapsed around the world, what happened?
Qs:Why was there a pre-break around 2000?
Qs:What is the speaker going to talk about?


...

Stage 1:../outputs/03_01_23_finance_course1/GQ.pickle has been saved


Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors


Stage 2:../outputs/03_01_23_finance_course1/QG+QA.pickle has been saved


100%|██████████| 60/60 [01:26<00:00,  1.44s/it]


Stage 3:../outputs/03_01_23_finance_course1/GQ+QA+GQ.pickle has been saved
Stage 4:../outputs/03_01_23_finance_course1/questions_full.txt has been saved
Stage 5:../outputs/03_01_23_finance_course1/questions.txt has been saved


In [2]:
%pdb on
try:
  from compute_question_and_answer import compute_question_and_answer
except:
  from compute_question_and_answer import compute_question_and_answer
import json
name = 'finance_course2'
sections_num_max = 'all'
JSON_path = '/home/ubuntu/Questions_generation/Financial Markets Course 2.json'
with open(JSON_path) as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
if sections_num_max!='all':
  sections = sections[:sections_num_max]
print(f'Limit questions to be from the {sections_num_max} first sections')
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

out = compute_question_and_answer(sections,org_text,save_file_name=name)


Automatic pdb calling has been turned ON
Limit questions to be from the all first sections


Token indices sequence length is longer than the specified maximum sequence length for this model (963 > 512). Running this sequence through the model will result in indexing errors


Abbreviation 	 Definition
{}


0it [00:00, ?it/s]

Total text shape is torch.Size([1, 376])
Qs:Who were some of these institutions that collapsed?
Qs:The speaker is going to talk about what in the context?
Qs:What happened after the stock market collapsed
Qs:How did this lecture begin?
Qs:When the stock market collapsed around the world, what happened?
Qs:Why was there a pre-break around 2000?
Qs:What is the speaker going to talk about?


1it [00:15, 15.45s/it]

Total text shape is torch.Size([1, 244])
Qs:Who built models that are built on theory fluid dynamics?
Qs:When was probability coined?
Qs:Why can't we predict hurricanes?
Qs:How do we build mathematical models of the outcomes of financial events?
Qs:What is the word probability in its present meaning?


2it [00:27, 13.22s/it]

Total text shape is torch.Size([1, 473])
Qs:Why can't you lose more than you put into an investment in finance?
Qs:Who has a limited liability economy?
Qs:Which of these is not a positive number?
Qs:How do you calculate the expected value for a random variable?


In [16]:
output = []
for i in np.unique(questions_df.section_n):
    section_output = {'question':[],'answer':[],'score':[]}
    for chunk in np.unique(questions_df.loc[questions_df.section_n==i,'section_n_chunk']):
        chunk_questions = questions_df.query(f'section_n_chunk=={chunk}')
        for _,q in chunk_questions.iterrows():
            if q['take']:
                section_output['question'].append(q.new_question)
                section_output['answer'].append(q['selected_ans'])
                section_output['score'].append(q['RQUGE'])
    output.append(section_output)

In [31]:
sections
dp = [len(x.split())>350 for x in sections]
print(dp)
len(list(filter(lambda x: x[1] is not None and dp[x[0]], enumerate(sections))))

[False, True]


1

In [11]:

text = ['Q:What was the derived uncertainty of the model?',
'A:4.1 minutes',
'Q:What is the uncertainty of the GBT model?',
'A:Gbt is the best performing model. the mean absolute error values in the table suggest that the performance is similar across all three algorithms. the derived uncertainty of the model was 4.1 minutes.',
'Q:What type of model is Barket-FM-DOSM?',
'A:Duration of surgery model',
'Q:What did the comparison lead to?',
'A:This comparison led to the conclusion that neither machine learning algorithms nor the dataset are the source of differences in the models performance. the major effector is that the set of features is the source.',
'Q:What is the main effector of differences in the model performance?',
'A:Dr. barket developed the model. the model is based on the surgery dataset. the main effector of such differences is the set of features.',
'Q:What is the most important feature?',
'A:Duration of surgery is the most important feature.',
'Q:3 out of 8 features selected are the same for which two methods?',
'A:Shap and pearson correlation.',
'Q:Where is Table 1 located?',
'A:Using machine learning techniques to develop supervised models that predict duration of surgery from features related to patients, physicians, and surgeries.',
'Q:What can be done with the predictions?',
'A:Use the dos value predicted by our model for surgery scheduling can decrease patient waiting time and maximize surgical staff idle time.']

%timeit mcq.sentence_model.encode(text)
%timeit [mcq.sentence_model.encode(t) for t in text]

10.7 ms ± 281 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
121 ms ± 894 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
import numpy as np
from collections import Counter
def check(args,prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    print(input_ids.shape)
    res = model.generate(input_ids, **args)
    return res['sequences'].shape
    output = tokenizer.batch_decode(res['sequences'], skip_special_tokens=True)

    output = [item.split("<sep>") for item in output]
    if all([len(cur_sen)==1 for cur_sen in output]):  
        output = [cur_sen[0] for cur_sen in output]
    if len(output)==1 and isinstance(output[0],list):
        output = output[0]
    output = [qs.strip() for qs in output]
    output = [*Counter(output)] #remove exactly similar questions if exist

    # Data originally sorted by probabilities, multiply be the penalities activated (`length_penalty` and/or `diversity_penalty`).
    # We want it to be sorted by the perplexity (PPL) score, the coherence of the sentence - probability normalized by the length of the sentence. 
    ppl = [np.exp(np.array(log_likelihoods.cpu()).mean() / np.array(len(cur_output.split())).mean()) for log_likelihoods,cur_output in zip(res['sequences_scores'],output)]
    sorted_idx  = np.argsort(ppl)
    return [output[id] for id in sorted_idx], [ppl[id] for id in sorted_idx]
generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}
from transformers import T5Tokenizer,T5ForConditionalGeneration
checkpoint = "google/flan-t5-large"#"IsaacBot/flan-t5-small-mfaq-finetuned-question-generation-context-only"# "google/flan-t5-xl"#"google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
COMES_sections = ['COMET is a recently proposed trainable neuralbased evaluation metric developed to assess the quality of Machine Translation systems. In this paper, we explore the usage of COMET for evaluating Text Summarization systems – despite being trained on multilingual MT outputs, it performs remarkably well in monolingual settings, when predicting summarization output quality. We introduce a variant of the model – COMES – trained on the annotated summarization outputs that uses MT data for pre-training. We examine its performance on several datasets with human judgments collected for different notions of summary quality, covering several domains and languages.',
                  ' Since manual annotation for any generative task is costly and time consuming, automatic metrics are commonly used to measure progress during training and compare output from independent systems. Metrics Shared Task collocated with the WMT workshop since 2008 (Callison-Burch et al., 2008) advances in the MT models performance are accompanied by a continuous development of new automatic metrics. They are robust to both domain shifts and changes in annotation style. One of the issues making research on summary evaluation metrics difficult is lack of standardized framework for collecting human judgments.  We propose a variant of the model – COMES2 – that uses the annotated MT data for pre-training and is capable of predicting several aspects of summary quality. We evaluate our approach (Section 4) on selected datasets with various annotated styles. We examine the applicability of the COMET metric by Rei et al. (2020) that is trained on the data and capable of directly regressing a quality score.',
                  ' For a comprehensive survey on the summary evaluation resources see Koto et al. and system output (Papineni et al., 2002; Lin, 2004). Over the years, a variety of metrics were proposed for this task – based on question answering, similarity between summary and reference embeddings.',
                  ' COMET is a trained metric that, based on semantic similarities between the translated and reference texts, learns to output a score that resembles the human perception of translation quality. COMET uses a pre-trained multilingual language model to extract representations for each of the input sequences, which are then pooled and concatenated, before being processed with a stack of feed-forward layers that outputs a single numerical value.']
print(check(generator_args,['generate question:'+i for i in COMES_sections]))

torch.Size([1, 5])


/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation/beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


torch.Size([8, 13])


In [20]:
h = tokenizer.encode('I love', return_tensors="pt")
print(tokenizer.decode(h[0]))

I love</s>


In [8]:
['generate question:'+i for i in COMES_sections]

['generate question:COMET is a recently proposed trainable neuralbased evaluation metric developed to assess the quality of Machine Translation systems. In this paper, we explore the usage of COMET for evaluating Text Summarization systems – despite being trained on multilingual MT outputs, it performs remarkably well in monolingual settings, when predicting summarization output quality. We introduce a variant of the model – COMES – trained on the annotated summarization outputs that uses MT data for pre-training. We examine its performance on several datasets with human judgments collected for different notions of summary quality, covering several domains and languages.',
 'generate question: Since manual annotation for any generative task is costly and time consuming, automatic metrics are commonly used to measure progress during training and compare output from independent systems. Metrics Shared Task collocated with the WMT workshop since 2008 (Callison-Burch et al., 2008) advances

In [7]:
'''from MCQ import flanT5MCQ
from datetime import datetime
generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}

answers_generator_args = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": 8,#10
    "length_penalty":0.2,
    #"length_penalty": 1.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[tokenizer.encode(['.'])],
    'top_p' :0.97,
    'diversity_penalty':float(8),
    'num_beam_groups':8,#10,
    "return_dict_in_generate" :True,
    'output_scores':True,
    "early_stopping": True,
    'num_return_sequences':5
}
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
dir_path = '../outputs/'+datetime.now().strftime("%d_%m_%y")+'_'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open(JSON_path) as f:
  result = json.load(f)

sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
sections = sections[:sections_num_max]
print(f'Limit questions to be from the {sections_num_max} first sections')
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]
min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
short_answers_generator_args = copy.deepcopy(answers_generator_args)
short_answers_generator_args["length_penalty"]=-0.6
mcq = flanT5MCQ(generator_args=generator_args,answers_generator_args=answers_generator_args,short_answers_generator_args=short_answers_generator_args)
'''
import pandas as pd
questions_df = pd.read_pickle('/home/ubuntu/Questions_generation/outputs/01_01_23_COMET2/GQ+QA+GQ.pickle')

questions_used = 'new_question'
questions_df['RQUGE'] = questions_df.apply(lambda x: mcq.rquge.scorer(x.text, x[questions_used], x.selected_ans)[0]['pred_score'] ,axis='columns')
questions_df = questions_df.sort_values('RQUGE',ascending=False).reset_index()
q_sim_mat,ans_sim_mat = mcq.find_similarity(questions_df[questions_used].to_list(),
                                            answers = questions_df['selected_ans'].to_list())
filter_idx = mcq.filter_questions(questions_df[questions_used].to_list(),
                                  q_sim_mat=q_sim_mat,ans_sim_mat=ans_sim_mat,
                                 similarity_thrs=0.75, n_thrs=20, return_index=True)
questions_df['use_question']=False
questions_df.loc[filter_idx,'use_question']=True
questions_df

,index,section_n,section_rank,text,question,question_ppl,answer_1,answer_2,answer_3,answer_4,short_answer_1,short_answer_2,short_answer_3,short_answer_4,generated_selected_ans,selected_ans,new_question,RQUGE,take
0,6,0.0,1.0,COMET is a recently proposed trainable neuralb...,What is the purpose of this paper?,0.053599,To evaluate text summarization systems using C...,Using COMET to evaluate text summarization sys...,An evaluation model for text summarization sys...,This paper introduces a variant of the COMET m...,to assess machine translation quality,Use COMET for text summarization evaluation,evaluate text summarization systems,To evaluate the quality of text summarization ...,to evaluate text summarization systems using c...,To evaluate text summarization systems using C...,What is the purpose of this paper?,4.803611,True
1,18,2.0,1.0,"COMET is a trained metric that, based on sema...",The COMET score is calculated by what method?,0.002099,Based on semantic similarities between the tra...,semantic similarities between the translated a...,The COMET score is calculated by a stack of fe...,pre-trained multilingual language model to ext...,stack of feed-forward layers,human perception of translation quality,pre-trained multilingual language model,semantic similarities between the translated a...,pre-trained multilingual language model to ext...,pre-trained multilingual language model to ext...,What does COMET use?,4.583428,True
2,7,1.0,1.0,Since manual annotation for any generative ta...,Annotation-based generative models: a new appr...,0.000009,"Using annotated datasets, we propose an automa...",In this paper we propose an annotated version ...,In this paper we propose an annotated version ...,COMES2 is a variant of the Metric Shared Task ...,an annotation-based generative model,Using annotated dataset to evaluate quality of...,Computational Evaluation of Summary Quality of...,Annotation-based generative models: a new appr...,in this paper we propose an annotated version ...,In this paper we propose an annotated version ...,How is the model evaluated?,4.370208,True
3,9,1.0,1.0,Since manual annotation for any generative ta...,Predicting summary evaluation metrics using an...,0.000261,In this paper we propose an automatic method f...,In this paper we propose an automatic method f...,"Using annotated MT data, we propose a variant ...",We propose a variant of the model – COMES2 – t...,Computational Modeling for Summary Evaluation,Predicting summary evaluation metrics using an...,Using annotated MT data to predict summary eva...,Predicting summary evaluation metrics using an...,in this paper we propose an automatic method f...,In this paper we propose an automatic method f...,How is the COMES2 model evaluated?,4.240835,True
4,14,1.0,1.0,Since manual annotation for any generative ta...,How to evaluate the quality of a generative mo...,0.007374,This paper presents an approach to evaluate th...,"Using annotated datasets, we propose COMES2 to...","In this paper, we propose COMES2 – a variant o...",COMES2 is a variant of the model that uses the...,CoMES2,the use of the COMET metrics,we introduce COMET2 and evaluate its performan...,COMET metric by Rei et al. (2020),comes2 is a variant of the model that uses the...,COMES2 is a variant of the model that uses the...,How is COMES2 evaluated?,4.126691,False
5,12,1.0,1.0,Since manual annotation for any generative ta...,Using annotated WMT data for pre-training and ...,0.000979,This paper presents an approach to automatical...,Using annotated WMT data for pre-training and ...,The COMET metric is a framework for evaluating...,COMES2 is a variant of the Metrics Shared Task...,Computational Evaluation of Summary Models usi...,Computational Evaluation of Summary Models,Using annotated WMT data for pre-training and ...,COMET2: Using annotated WMT data for pre-train...,comes2 is a variant of the metrics shared task...,COMES2 is a variant of the Metrics Shared Task...,How is COMES2 evaluated?,4.089975,

In [8]:
#!pip install pytextrank
#!pip install lmqg
from lmqg import TransformersQG
# initialize model
del model
model = TransformersQG(language='en', model='lmqg/t5-large-squad-qg-ae')
# paragraph to generate pairs of question and answer
#context = "William Turner was an English painter who specialised in watercolour landscapes. He is often known as William Turner of Oxford or just Turner of Oxford to distinguish him from his contemporary, J. M. W. Turner. Many of Turner's paintings depicted the countryside around Oxford. One of his best known pictures is a view of the city of Oxford from Hinksey Hill."
# model prediction
question_answer = model.generate_qa(sections[0])
# the output is a list of tuple (question, answer)
print(question_answer)

TypeError: __init__() got an unexpected keyword argument 'device'

> /tmp/ipykernel_39270/3981189669.py(6)<module>()
      4 # initialize model
      5 del model
----> 6 model = TransformersQG(language='en', model='lmqg/t5-large-squad-qg-ae',device='cpu')
      7 # paragraph to generate pairs of question and answer
      8 #context = "William Turner was an English painter who specialised in watercolour landscapes. He is often known as William Turner of Oxford or just Turner of Oxford to distinguish him from his contemporary, J. M. W. Turner. Many of Turner's paintings depicted the countryside around Oxford. One of his best known pictures is a view of the city of Oxford from Hinksey Hill."



In [11]:

filter_idx = mcq.filter_questions(questions_df[questions_used].to_list(),
                                 mcq.find_similarity(questions_df[questions_used].to_list()),
                                 similarity_thrs=0.75, n_thrs=20, return_index=True)
filter_idx

[0, 2, 4, 5, 10, 13, 14, 15, 21, 22]

In [8]:
from negspacy.negation import Negex

nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe('sentencizer')
nlp.add_pipe(Negex(nlp))

TypeError: __init__() missing 5 required positional arguments: 'name', 'neg_termset', 'ent_types', 'extension_name', and 'chunk_prefix'

> /tmp/ipykernel_3955/103320592.py(5)<module>()
      1 from negspacy.negation import Negex
      2 
      3 nlp = spacy.load("en_core_sci_sm")
      4 nlp.add_pipe('sentencizer')
----> 5 nlp.add_pipe(Negex(nlp))



In [2]:
import spacy
from negspacy.negation import Negex
#!pip install spacy
#!python3 -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex", config={"ent_types":["PERSON","ORG"]})

doc = nlp("She does not like Steve Jobs but likes Apple products.")
for e in doc.ents:
    print(e.text, e._.negex)

/home/ubuntu/.local/lib/python3.8/site-packages/spacy/util.py:885: UserWarning: [W094] Model 'en_core_web_sm' (2.2.0) specifies an under-constrained spaCy version requirement: >=2.2.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.4.3,<3.5.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /home/ubuntu/.local/lib/python3.8/site-packages/en_core_web_sm/en_core_web_sm-2.2.0/config.cfg

In [12]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

     |████████████████████████████████| 13.7 MB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 24.4 MB/s eta 0:00:01
     |████████████████████████████████| 671 kB 72.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.7 MB 75.4 MB/s eta 0:00:01
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Created wheel for en-core-web-sm: filename=en_core_web_sm-3.0.0-py3-none-any.whl size=13704312 sha256=9005cfa5fcba0510ddcb2d3635fcb34905ac67596c410e72d67b8cc361e6d053
  Stored in directory: /home/ubuntu/.cache/pip/wheels/8b/21/c1/257748af7399fdaf1b2afc39c92fb839c436f42e67b656ff7e
Successfully built en-core-web-sm
ERROR: scispacy 0.5.1 has requirement spacy<3.5.0,>=3.4.0, but you'll have spacy 3.0.9 which is incompatible.
ERROR: en-core-sci-sm 0.5.1 has requirement spacy<3.5.0,>=3.4.1, but you'll have spacy 3.0.9 which is incompatible.
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully

In [13]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Aren’t you coming? Doesn’t he understand? Are you not coming? Does he not understand? Which of the following did not occur? Does Jeff is a real person? Name something David hadn't reveal in his vacation?")
any([token.dep_=='neg' for token in doc])

True

In [7]:
!python3 -m spacy validate

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.8/runpy.py", line 144, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.8/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/spacy/__init__.py", line 15, in <module>
    from .cli.info import info  # noqa: F401
  File "/home/ubuntu/.local/lib/python3.8/site-packages/spacy/cli/__init__.py", line 17, in <module>
    from .debug_diff import debug_diff  # noqa: F401
  File "/home/ubuntu/.local/lib/python3.8/site-packages/spacy/cli/debug_diff.py"

In [2]:
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
name = 'Financial_Markets_Course2_flanT5L'

dir_path = '../outputs/'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open('/home/ubuntu/Questions_generation/Financial Markets Course 2.json') as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
from MCQ import flanT5MCQ

generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}

answers_generator_args = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": 8,#10
    "length_penalty":0.2,
    #"length_penalty": 1.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[tokenizer.encode(['.'])],
    'top_p' :0.97,
    'diversity_penalty':float(8),
    'num_beam_groups':8,#10,
    "return_dict_in_generate" :True,
    'output_scores':True,
    "early_stopping": True,
    'num_return_sequences':5
}
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
name = 'Financial_Markets_Course2_flanT5L'

dir_path = '../outputs/'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open('/home/ubuntu/Questions_generation/Financial Markets Course 2.json') as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
short_answers_generator_args = copy.deepcopy(answers_generator_args)
short_answers_generator_args["length_penalty"]=-0.6
mcq = flanT5MCQ(generator_args=generator_args,answers_generator_args=answers_generator_args,short_answers_generator_args=short_answers_generator_args)

In [7]:
import re
def is_similar(string, patterns):
    # compile the regex pattern to remove spaces and punctuation marks
    string = string.replace('_','')
    pattern = r'[^\w\s]'
    regex = re.compile(pattern)

    # remove spaces and punctuation marks from the input string and the patterns
    string = regex.sub('', string)
    patterns = [regex.sub('', p) for p in patterns]

    # check if the input string is exactly the same as one of the patterns
    return string.lower().strip() in patterns
answers_black_list =['we','they','the authors','authors','author','the author','you','you are','we are',
                                'the speaker','speaker','the lecturer','lecturer',
                                'he','he is','she','she is',
                                  'I','these','those','they are','we do','it','is is']
inputs = ['we','We','we...','They!','  Go','THEY  _ ']
[is_similar(ak,answers_black_list) for ak in inputs]

[True, True, True, True, False, False]

In [20]:
def text_slicer( text):
        print(len(mcq.tokenizer(text, return_tensors="pt")["input_ids"][0]))
        if len(mcq.tokenizer(text, return_tensors="pt")["input_ids"][0]) <= mcq.tokenizer.model_max_length :
            return [text]
        chunks = []
        chunk = []
        length = 0
        for sentence in nltk.tokenize.sent_tokenize(text):
            _len = len(mcq.tokenizer.tokenize(sentence))
            if length + _len <= mcq.tokenizer.model_max_length :
                length += _len
                chunk.append(sentence)
            elif not chunk:
                # Can a sentence be applicable for splitting on to chunks?
                chunks.append(sentence.strip())
                length = 0
            else:
                chunks.append(' '.join(chunk).strip())
                chunk = [sentence]
                length = _len
        if chunk:
            chunks.append(' '.join(chunk).strip())
        return chunks
sections_chunks = []
sections_n = []
for i,cur_section in enumerate(sections):
    cur_section_chunks = text_slicer(cur_section)
    n_chunks = len(cur_section_chunks)
    sections_chunks.extend(cur_section_chunks)
    if n_chunks==1:
        sections_n.append(float(i))
    else:
        sections_n.extend([i+0.1*k for k in range(n_chunks)])
print('='*100)
[text_slicer(cur_section) for cur_section in sections_chunks]

376
244
963
534
1167
664
376
244
512
452
473
62
510
485
174
489
176


[[" The crisis began with bubbles in the stock market, housing market, and also in the commodities market. It's a financial crisis that's bigger than any since the Great Depression of the 1930's. There's many different ways of thinking about a crisis like this. And I wanted to focus on one way that people think about it in terms of probability models. So, that's not the only way, it's not necessarily my favorite way. Excuse my cold. I didn't bring any water. I hope I make it through this lecture.  There was a pre-break around 2000 when the stock market collapsed around the world. But then they came back again after 2003 and they were on another boom, like a roller coaster ride. That's the narrative story. And then, what happened is, we see a bunch of institutional collapses. We saw bank failures in the U.S. and then, we saw international cooperation to prevent this from spreading like a disease. So, we had governments all over the world bailing out their banks and other companies. That

In [19]:
len(mcq.tokenizer(sections, return_tensors="pt")["input_ids"][0]) 

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True